<a href="https://colab.research.google.com/github/saiharitha1310/sai/blob/main/Sequential_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
from PIL import Image

np.random.seed(42)
from sklearn.metrics import confusion_matrix

import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout,MaxPool2D,Flatten,Conv2D,BatchNormalization
from sklearn.model_selection import train_test_split
from scipy import stats
from sklearn.preprocessing import LabelEncoder


from google.colab import drive
drive.mount('/content/drive')

skin_df=pd.read_csv('/content/drive/MyDrive/HAM/HAM10000_metadata.csv')


size=32
le=LabelEncoder()
le.fit(skin_df['dx'])
LabelEncoder()
print(list(le.classes_))
skin_df['label']=le.transform(skin_df["dx"])
print(skin_df.sample(10))


n_samples=800
df_0_downsampled=resample(df_0,replace=True,n_samples=n_samples,random_state=42)
df_1_downsampled=resample(df_1,replace=True,n_samples=n_samples,random_state=42)
df_2_downsampled=resample(df_2,replace=True,n_samples=n_samples,random_state=42)
df_3_downsampled=resample(df_3,replace=True,n_samples=n_samples,random_state=42)
df_4_downsampled=resample(df_4,replace=True,n_samples=n_samples,random_state=42)
df_5_downsampled=resample(df_5,replace=True,n_samples=n_samples,random_state=42)
df_6_downsampled=resample(df_6,replace=True,n_samples=n_samples,random_state=42)
skin_df_balanced=pd.concat([df_0_downsampled,df_1_downsampled,df_2_downsampled,df_3_downsampled,df_4_downsampled,df_5_downsampled,df_6_downsampled])
print(skin_df_balanced['label'].value_counts())


image_path = {os.path.splitext(os.path.basename(x))[0]: x
                     for x in glob(os.path.join('/content/drive/MyDrive/HAM/','*', '*.jpg'))}
skin_df_balanced['path']=skin_df_balanced['image_id'].map(image_path.get)
skin_df_balanced['image']=skin_df_balanced['path'].map(lambda x: np.asarray(Image.open(x).resize((32,32))))


n_samples=5
fig,m_axs=plt.subplots(7,n_samples,figsize=(4*n_samples,3*7))
for n_axs, (type_name,type_rows) in zip(m_axs,skin_df_balanced.sort_values(['dx']).groupby('dx')):
  n_axs[0].set_title(type_name)
  for c_ax,(_,c_row) in zip(n_axs,type_rows.sample(n_samples,random_state=1234).iterrows()):
    c_ax.imshow(c_row['image'])
    c_ax.axis('off')


X=np.asarray(skin_df_balanced['image'].tolist())
X=X/255
y=skin_df_balanced['label']
y=to_categorical(y,num_classes=7)
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=42)


num_classes=7
model=Sequential()
model.add(Conv2D(256,kernel_size=(3,3),activation='relu',padding='same',input_shape=(32,32,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128,kernel_size=(3,3),activation='relu',padding='same'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(64,kernel_size=(3,3),activation='relu',padding='same'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(32))
model.add(Dense(num_classes,activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


batch_size=11
epochs=5
history=model.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,verbose=2,validation_data=(x_test,y_test))

score=model.evaluate(x_test,y_test,verbose=0)
print('Test loss:',score[0])
print('Test accuracy:',score[1])